In [ ]:
#| include: false
#| echo: false
! [ -e /content ] && pip install -Uqq gingado nbdev # install or upgrade gingado on colab

In [ ]:
#| default_exp augmentation
#| include: false
%load_ext autoreload
%autoreload 2

In [ ]:
#| include: false
from nbdev.showdoc import show_doc

`gingado` provides data augmentation functionalities that can help users to augment their datasets with a time series dimension. This can be done both on a stand-alone basis as the user incorporates new data on top of the original dataset, or as part of a `scikit-learn` [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) that also includes other steps like data transformation and model estimation.

# Data augmentation with SDMX

The **S**tatistical **D**ata and **M**etadata e**X**change (SDMX) is an ISO standard comprising:

* technical standards

* statistical guidelines, including cross-domain concepts and codelists

* an IT architecture and tools

SDMX is sponsored by the Bank for International Settlements, European Central Bank, Eurostat, International Monetary Fund, Organisation for Economic Co-operation and Development, United Nations, and World Bank Group.

More information about the SDMX is available on its [webpage](http://sdmx.org).

`gingado` uses SDMX to augment user datasets through the transformer `AugmentSDMX`.

In [ ]:
#| include: false
#| export

from gingado.utils import load_SDMX_data
import numpy as np
import pandas as pd
import pandasdmx as sdmx
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
from sklearn.feature_selection import VarianceThreshold

In [ ]:
#| include: false
#| export

class AugmentSDMX(BaseEstimator, TransformerMixin):
    "A transformer that augments a dataset using SDMX"
    
    InputIndexMessage = "The dataset to be augmented must have a row index with the date/time information"
    def _format_string(self):
        return "%Y-%m-%d" if self.data_freq_ == 'D' else "%Y-%m" if self.data_freq_ == 'M' else "%Y"
    
    def _get_dates(self):
        fstr = self._format_string()
        return {
            "startPeriod": min(self.index_).strftime(fstr),
            "endPeriod": max(self.index_).strftime(fstr),
        }

    def _transform(self, X, training=True):
        df = load_SDMX_data(sources=self.sources, keys=self.keys_, params=self.params_, verbose=self.verbose)
        if df is None:
            return X

        if training:
            # test that the dataset `X` has the same dimension as the one
            # used during training, which is an evidence they are the same
            n_samples_in_transform, n_features_in_transform = X.shape
            if n_samples_in_transform != self.n_samples_in_ or n_features_in_transform != self.n_features_in_:
                raise ValueError("The `X` passed to the transform() method must be compatible with the `X` used by the fit() method.")
            # during testing, we don't want the possibility of a different
            # set of columns being retained by virtue of different dynamics
            # in both datasets. For example, if a feature is included in the
            # training but during the test dates the variable didn't move, it
            # should not be subject to the test below so that it is still
            # included in the fitted data.
            self.features_stay_ = df.columns
            self.features_removed_ = None
            merge_df = True
            if self.variance_threshold:
                df_temp = df.dropna(axis=0, how='any')
                feat_sel = VarianceThreshold(threshold=self.variance_threshold)
                try:
                    feat_sel.fit(df)
                    self.features_stay_ = df.columns[feat_sel.get_support()]
                    self.features_removed_ = df.columns[~feat_sel.get_support()]
                    df = df.iloc[:, feat_sel.get_support()]
                    df.columns = feat_sel.get_feature_names_out()
                except ValueError as e:
                    print("No columns added to original data because " + str(e).lower())
                    merge_df = False
            df.dropna(axis=0, how='all', inplace=True)
            df.dropna(axis=1, how='all', inplace=True)  
            if merge_df:
                X = pd.merge(left=X, right=df, how='left', left_index=True, right_on='TIME_PERIOD')      
        
        if training is False: # if True, `X` would already have merged with `df` (or not if no added column)
            X = pd.merge(left=X, right=df, how='left', left_index=True, right_on='TIME_PERIOD')
        if 'TIME_PERIOD' in X.columns:
            X.drop(columns='TIME_PERIOD', inplace=True)
        if self.propagate_last_known_value:
            X.fillna(method="ffill", inplace=True)
        if self.fillna is not None:
            X.fillna(self.fillna)
        if training:
            X.index = self.index_
        return X

    def __init__(
        self,
        sources:dict={'BIS': 'WS_CBPOL_D'}, # A dictionary with sources as keys and dataflows as values
        variance_threshold:float|None=None, # If None (default), all variables are kept. Otherwise, variables that have a lower variance through time are removed
        propagate_last_known_value:bool=True, # Whether the last value that is not NA should be propagated to the following dates
        fillna:float|int = 0, # Value to use to fill missing data
        verbose:bool=True # Whether to inform the user as the process progresses
        ):
        self.sources = sources
        self.variance_threshold = variance_threshold
        self.propagate_last_known_value = propagate_last_known_value
        self.fillna = fillna
        self.verbose = verbose

    def fit(
            self,
            X:pd.Series|pd.DataFrame, # Data having an index of `datetime` type
            y:None=None # `y` is kept as argument for API consistency only
        ): # A fitted version of the same AugmentSDMX instance
        "Fits instance of AugmentSDMX to `X`, learning its time series frequency"
        
        try:
            self.data_freq_ = X.index.to_series().diff().min().resolution_string
        except AttributeError:
            print(self.InputIndexMessage)
            raise
        self.index_ = X.index
        self.keys_ = {'FREQ': self.data_freq_}
        X = self._validate_data(X)

        # this variable below is only included to test for consistency \
        # if `fit` and `transform` are called separately with the same `X`
        self.n_samples_in_ = X.shape[0]

        return self

    def transform(
            self,
            X:pd.Series|pd.DataFrame, # Data having an index of `datetime` type
            y:None=None, # `y` is kept as argument for API consistency only
            training:bool=False # `True` if `transform` is called during training, `False` (default) if called during testing
        ): # `X` augmented with data from SDMX
        "Transforms input dataset `X` by adding the requested data using SDMX"
        check_is_fitted(self)
        self.params_ = self._get_dates()
        idx = X.index
        transf_X = self._transform(X, training=training)
        transf_X.index = idx
        return transf_X

    def fit_transform(
            self, 
            X:pd.Series|pd.DataFrame, # Data having an index of `datetime` type
            y:None=None # `y` is kept as argument for API consistency only
            ) -> np.ndarray: # `X` augmented with data from SDMX with the same number of samples but more columns
        "Fit to data, then transform it."
        
        self.fit(X)
        self.params_ = self._get_dates()
        return self.transform(X, training=True)

In [ ]:
show_doc(AugmentSDMX)

---

[source](https://github.com/dkgaraujo/gingado/blob/main/gingado/augmentation.py#L16){target="_blank" style="float:right; font-size:smaller"}

### AugmentSDMX

>      AugmentSDMX (sources:dict={'BIS': 'WS_CBPOL_D'},
>                   variance_threshold:float|None=None,
>                   propagate_last_known_value:bool=True, fillna:float|int=0,
>                   verbose:bool=True)

A transformer that augments a dataset using SDMX

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| sources | dict | {'BIS': 'WS_CBPOL_D'} | A dictionary with sources as keys and dataflows as values |
| variance_threshold | float \| None | None | If None (default), all variables are kept. Otherwise, variables that have a lower variance through time are removed |
| propagate_last_known_value | bool | True | Whether the last value that is not NA should be propagated to the following dates |
| fillna | float \| int | 0 | Value to use to fill missing data |
| verbose | bool | True | Whether to inform the user as the process progresses |

For example, the code is a simple illustration of `AugmentSDMX` augmentation under two scenarios: without a variance threshold (ie, including all data regardless if they are constants) or with a relatively high variance threshold (such that no data is actually added).

In both cases, the object is using the default data flow, which is the daily series of monetary policy rates set by central banks.

These `AugmentSDMX` objects are used to augment a data frame with simulated data for illustrative purposes. In real life, this data would be the user's original data.

In [ ]:
rng = np.random.default_rng(seed=42)

periods = 15
idx = pd.date_range(freq='d', start='2020-01-01', periods=periods)
orig_data = pd.DataFrame({'orig_col': rng.standard_normal(periods)}, index=idx)
orig_data.head()

,orig_col
2020-01-01,0.304717
2020-01-02,-1.039984
2020-01-03,0.750451
2020-01-04,0.940565
2020-01-05,-1.951035


In [ ]:
aug_NoVarThresh = AugmentSDMX(variance_threshold=None)
aug_data = aug_NoVarThresh.fit_transform(orig_data)
aug_data

Querying data from BIS's dataflow 'WS_CBPOL_D' - Policy rates daily...


,orig_col,BIS__WS_CBPOL_D_D__AR,BIS__WS_CBPOL_D_D__AU,BIS__WS_CBPOL_D_D__BR,BIS__WS_CBPOL_D_D__CA,BIS__WS_CBPOL_D_D__CH,BIS__WS_CBPOL_D_D__CL,BIS__WS_CBPOL_D_D__CN,BIS__WS_CBPOL_D_D__CO,BIS__WS_CBPOL_D_D__CZ,...,BIS__WS_CBPOL_D_D__RO,BIS__WS_CBPOL_D_D__RS,BIS__WS_CBPOL_D_D__RU,BIS__WS_CBPOL_D_D__SA,BIS__WS_CBPOL_D_D__SE,BIS__WS_CBPOL_D_D__TH,BIS__WS_CBPOL_D_D__TR,BIS__WS_CBPOL_D_D__US,BIS__WS_CBPOL_D_D__XM,BIS__WS_CBPOL_D_D__ZA
2020-01-01,0.304717,55.0,NaN,NaN,1.75,-0.75,NaN,4.15,4.25,NaN,...,NaN,2.25,NaN,2.25,NaN,NaN,NaN,1.625,0.0,NaN
2020-01-02,-1.039984,55.0,0.75,4.5,1.75,-0.75,1.75,4.15,4.25,2.0,...,NaN,2.25,NaN,2.25,-0.25,1.25,12.0,1.625,0.0,6.5
2020-01-03,0.750451,55.0,0.75,4.5,1.75,-0.75,1.75,4.15,4.25,2.0,...,2.5,2.25,NaN,2.25,-0.25,1.25,12.0,1.625,0.0,6.5
2020-01-04,0.940565,55.0,0.75,4.5,1.75,-0.75,1.75,4.15,4.25,2.0,...,2.5,2.25,NaN,2.25,-0.25,1.25,12.0,1.625,0.0,6.5
2020-01-05,-1.951035,55.0,0.75,4.5,1.75,-0.75,1.75,4.15,4.25,2.0,...,2.5,2.25,NaN,2.25,-0.25,1.25,12.0,1.625,0.0,6.5
2020-01-06,-1.302180,55.0,0.75,4.5,1.75,-0.75,1.75,4.15,4.25,2.0,...,2.5,2.25,6.25,2.25,-0.25,1.25,12.0,1.625,0.0,6.5
2020-01-07,0.127840,55.0,0.75,4.5,1.75,-0.75,1.75,4.15,4.25,2.0,...,2.5,2.25,6.25,2.25,-0.25,1.25,12.0,1.625,0.0,6.5
2020-01-08,-0.316243,55.0,0.75,4.5,1.75,-0.75,1.75,4.15,4.25,2.0,...,2.5,2.25,6.25,2.25,0.00,1.25,12.0,1.625,0.0,6.5
2020-01-09,-0.016801,55.0,0.75,4.5,1.75,-0.75,1.75,4.15,4.25,2.0,...,2.5,2.25,6.25,2.25,0.00,1.25,12.0,1.625,0.0,6.5
2020-01-10,-0.853044,52.0,0.75,4.5,1.75,-0.75,1.75,4.15,4.25,2.0,...,2.5,2.25,6.25,2.25,0.00,1.25,12.0,1.625,0.0,6.5


In [ ]:
aug_StrictVarThresh = AugmentSDMX(variance_threshold=10)
aug_data = aug_StrictVarThresh.fit_transform(orig_data)
aug_data

Querying data from BIS's dataflow 'WS_CBPOL_D' - Policy rates daily...
No columns added to original data because no feature in x meets the variance threshold 10.00000


/Users/douglasaraujo/Coding/.venv_gingado/lib/python3.10/site-packages/sklearn/feature_selection/_variance_threshold.py:104: RuntimeWarning: Degrees of freedom <= 0 for slice.
  self.variances_ = np.nanvar(X, axis=0)


,orig_col
2020-01-01,0.304717
2020-01-02,-1.039984
2020-01-03,0.750451
2020-01-04,0.940565
2020-01-05,-1.951035
2020-01-06,-1.302180
2020-01-07,0.127840
2020-01-08,-0.316243
2020-01-09,-0.016801
2020-01-10,-0.853044


In [ ]:
show_doc(AugmentSDMX.fit)

---

[source](https://github.com/dkgaraujo/gingado/blob/main/gingado/augmentation.py#L76){target="_blank" style="float:right; font-size:smaller"}

### AugmentSDMX.fit

>      AugmentSDMX.fit (X:pandas.core.series.Series|pandas.core.frame.DataFrame,
>                       y:None=None)

Fits instance of AugmentSDMX to `X`, learning its time series frequency

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| X | pandas.core.series.Series \| pandas.core.frame.DataFrame |  | Data having an index of `datetime` type |
| y | None | None | `y` is kept as argument for API consistency only |

In [ ]:
show_doc(AugmentSDMX.transform)

---

[source](https://github.com/dkgaraujo/gingado/blob/main/gingado/augmentation.py#L98){target="_blank" style="float:right; font-size:smaller"}

### AugmentSDMX.transform

>      AugmentSDMX.transform
>                             (X:pandas.core.series.Series|pandas.core.frame.Dat
>                             aFrame, y:None=None, training:bool=False)

Transforms input dataset `X` by adding the requested data using SDMX

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| X | pandas.core.series.Series \| pandas.core.frame.DataFrame |  | Data having an index of `datetime` type |
| y | None | None | `y` is kept as argument for API consistency only |
| training | bool | False | `True` if `transform` is called during training, `False` (default) if called during testing |

In [ ]:
show_doc(AugmentSDMX.fit_transform)

---

[source](https://github.com/dkgaraujo/gingado/blob/main/gingado/augmentation.py#L112){target="_blank" style="float:right; font-size:smaller"}

### AugmentSDMX.fit_transform

>      AugmentSDMX.fit_transform
>                                 (X:pandas.core.series.Series|pandas.core.frame
>                                 .DataFrame, y:None=None)

Fit to data, then transform it.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| X | pandas.core.series.Series \| pandas.core.frame.DataFrame |  | Data having an index of `datetime` type |
| y | None | None | `y` is kept as argument for API consistency only |
| **Returns** | **ndarray** |  | **`X` augmented with data from SDMX with the same number of samples but more columns** |

### Compatibility with `scikit-learn`

As mentioned above, `gingado`'s transformers are built to be compatible with `scikit-learn`. The code below demonstrates this compatibility.

First, we create the example dataset. In this case, it comprises the daily foreign exchange rate of selected currencies to the Euro. The Brazilian Real (BRL) is chosen for this example as the dependent variable.

In [ ]:
from gingado.utils import load_SDMX_data, Lag
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
X = load_SDMX_data(
    sources={'ECB': 'EXR'}, 
    keys={'FREQ': 'D', 'CURRENCY': ['EUR', 'AUD', 'BRL', 'CAD', 'CHF', 'GBP', 'JPY', 'SGD', 'USD']},
    params={"startPeriod": 2003}
    )
# drop rows with empty values
X.dropna(inplace=True)
# adjust column names in this simple example for ease of understanding:
# remove parts related to source and dataflow names
X.columns = X.columns.str.replace("ECB__EXR_D__", "").str.replace("__EUR__SP00__A", "")
X = Lag(lags=1, jump=0, keep_contemporaneous_X=True).fit_transform(X)
y = X.pop('BRL')
# retain only the lagged variables in the X variable
X = X[X.columns[X.columns.str.contains('_lag_')]]

Querying data from ECB's dataflow 'EXR' - Exchange Rates...


2023-06-18 11:29:19,756 pandasdmx.reader.sdmxml - INFO: Use supplied dsd=… argument for non–structure-specific message


In [ ]:
X_train, X_test = X.iloc[:-1], X.tail(1)
y_train, y_test = y.iloc[:-1], y.tail(1)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((5239, 8), (5239,), (1, 8), (1,))

Next, the data augmentation object provided by `gingado` adds more data. In this case, for brevity only one dataflow from one source is listed. If users want to add more SDMX sources, simply add more keys to the dictionary. And if users want data from all dataflows from a given source provided the keys and parameters such as frequency and dates match, the value should be set to `'all'`, as in `{'ECB': ['CISS'], 'BIS': 'all'}`.

In [ ]:
test_src = {'ECB': ['CISS'], 'BIS': ['WS_CBPOL_D']}

X_train__fit_transform = AugmentSDMX(sources=test_src).fit_transform(X=X_train)
X_train__fit_then_transform = AugmentSDMX(sources=test_src).fit(X=X_train).transform(X=X_train, training=True)

assert X_train__fit_transform.shape == X_train__fit_then_transform.shape

Querying data from ECB's dataflow 'CISS' - Composite Indicator of Systemic Stress...


2023-06-18 11:31:53,806 pandasdmx.reader.sdmxml - INFO: Use supplied dsd=… argument for non–structure-specific message


Querying data from BIS's dataflow 'WS_CBPOL_D' - Policy rates daily...
Querying data from ECB's dataflow 'CISS' - Composite Indicator of Systemic Stress...


2023-06-18 11:33:30,655 pandasdmx.reader.sdmxml - INFO: Use supplied dsd=… argument for non–structure-specific message


Querying data from BIS's dataflow 'WS_CBPOL_D' - Policy rates daily...


This is the dataset now after this particular augmentation:

In [ ]:
print(f"No of columns: {len(X_train__fit_transform.columns)} {X_train__fit_transform.columns}")
X_train__fit_transform

No of columns: 69 Index(['AUD_lag_1', 'BRL_lag_1', 'CAD_lag_1', 'CHF_lag_1', 'GBP_lag_1',
       'JPY_lag_1', 'SGD_lag_1', 'USD_lag_1',
       'ECB__CISS_D__AT__Z0Z__4F__EC__SS_CIN__IDX',
       'ECB__CISS_D__BE__Z0Z__4F__EC__SS_CIN__IDX',
       'ECB__CISS_D__CN__Z0Z__4F__EC__SS_CIN__IDX',
       'ECB__CISS_D__DE__Z0Z__4F__EC__SS_CIN__IDX',
       'ECB__CISS_D__ES__Z0Z__4F__EC__SS_CIN__IDX',
       'ECB__CISS_D__FI__Z0Z__4F__EC__SS_CIN__IDX',
       'ECB__CISS_D__FR__Z0Z__4F__EC__SS_CIN__IDX',
       'ECB__CISS_D__GB__Z0Z__4F__EC__SS_CIN__IDX',
       'ECB__CISS_D__IE__Z0Z__4F__EC__SS_CIN__IDX',
       'ECB__CISS_D__IT__Z0Z__4F__EC__SS_CIN__IDX',
       'ECB__CISS_D__NL__Z0Z__4F__EC__SS_CIN__IDX',
       'ECB__CISS_D__PT__Z0Z__4F__EC__SS_CIN__IDX',
       'ECB__CISS_D__U2__Z0Z__4F__EC__SS_BM__CON',
       'ECB__CISS_D__U2__Z0Z__4F__EC__SS_CI__IDX',
       'ECB__CISS_D__U2__Z0Z__4F__EC__SS_CIN__IDX',
       'ECB__CISS_D__U2__Z0Z__4F__EC__SS_CO__CON',
       'ECB__CISS_D__U2__Z0Z__4F__E

,AUD_lag_1,BRL_lag_1,CAD_lag_1,CHF_lag_1,GBP_lag_1,JPY_lag_1,SGD_lag_1,USD_lag_1,ECB__CISS_D__AT__Z0Z__4F__EC__SS_CIN__IDX,ECB__CISS_D__BE__Z0Z__4F__EC__SS_CIN__IDX,...,BIS__WS_CBPOL_D_D__RO,BIS__WS_CBPOL_D_D__RS,BIS__WS_CBPOL_D_D__RU,BIS__WS_CBPOL_D_D__SA,BIS__WS_CBPOL_D_D__SE,BIS__WS_CBPOL_D_D__TH,BIS__WS_CBPOL_D_D__TR,BIS__WS_CBPOL_D_D__US,BIS__WS_CBPOL_D_D__XM,BIS__WS_CBPOL_D_D__ZA
TIME_PERIOD,,,,,,,,,,,,,,,,,,,,,
2003-01-03,1.8554,3.6770,1.6422,1.4528,0.65200,124.40,1.8188,1.0446,0.021899,0.043292,...,NaN,9.5,NaN,NaN,3.75,1.75,44.0,1.250,2.75,13.50
2003-01-06,1.8440,3.6112,1.6264,1.4555,0.65000,124.56,1.8132,1.0392,0.020801,0.039924,...,19.75,9.5,NaN,2.00,3.75,1.75,44.0,1.250,2.75,13.50
2003-01-07,1.8281,3.5145,1.6383,1.4563,0.64950,124.40,1.8210,1.0488,0.019738,0.038084,...,19.75,9.5,NaN,2.00,3.75,1.75,44.0,1.250,2.75,13.50
2003-01-08,1.8160,3.5139,1.6257,1.4565,0.64960,124.82,1.8155,1.0425,0.019947,0.040338,...,19.75,9.5,21.0,2.00,3.75,1.75,44.0,1.250,2.75,13.50
2003-01-09,1.8132,3.4405,1.6231,1.4586,0.64950,124.90,1.8102,1.0377,0.017026,0.040535,...,19.75,9.5,21.0,2.00,3.75,1.75,44.0,1.250,2.75,13.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-09,1.6061,5.2866,1.4342,0.9751,0.86113,149.98,1.4460,1.0737,0.167185,0.124553,...,7.00,6.0,7.5,5.75,3.50,2.00,8.5,5.125,3.75,8.25
2023-06-12,1.6023,5.2965,1.4362,0.9716,0.85795,150.24,1.4480,1.0780,0.171139,0.123640,...,7.00,6.0,7.5,5.75,3.50,2.00,8.5,5.125,3.75,8.25
2023-06-13,1.5920,5.2549,1.4357,0.9751,0.85678,150.03,1.4457,1.0765,0.164665,0.118228,...,7.00,6.0,7.5,5.75,3.50,2.00,8.5,5.125,3.75,8.25


`AugmentSDMX` can also be part of a `Pipeline` object, which minimises operational errors during modelling and avoids using testing data during training:

In [ ]:
from gingado.augmentation import AugmentSDMX
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
pipeline = Pipeline([
    ('augmentation', AugmentSDMX(sources={'BIS': 'WS_CBPOL_D'})),
    ('imp', IterativeImputer(max_iter=10)),
    ('forest', RandomForestRegressor())
], verbose=True)

### Tuning the data augmentation to enhance model performance

And since `AugmentSDMX` can be included in a `Pipeline`, it can also be fine-tuned by parameter search techniques (such as grid search), further helping users make the best of available data to enhance performance of their models.

::: {.callout-tip}
Users can cache the data augmentation step to avoid repeating potentially lengthy data downloads. See the `memory` argument in the [`sklearn.pipeline.Pipeline` documentation](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).
:::

In [ ]:
grid = GridSearchCV(
    estimator=pipeline,
    param_grid={
        'augmentation': ['passthrough', AugmentSDMX(sources={'ECB': 'CISS'})]
    },
    verbose=2,
    cv=TimeSeriesSplit(n_splits=2)
    )

y_pred_grid = grid.fit(X_train, y_train).predict(X_test)

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[Pipeline] ...... (step 1 of 3) Processing augmentation, total=   0.0s
[Pipeline] ............... (step 2 of 3) Processing imp, total=   0.0s
[Pipeline] ............ (step 3 of 3) Processing forest, total=   0.7s
[CV] END ...........................augmentation=passthrough; total time=   0.7s
[Pipeline] ...... (step 1 of 3) Processing augmentation, total=   0.0s
[Pipeline] ............... (step 2 of 3) Processing imp, total=   0.0s
[Pipeline] ............ (step 3 of 3) Processing forest, total=   1.3s
[CV] END ...........................augmentation=passthrough; total time=   1.3s


2023-06-18 11:37:46,240 pandasdmx.reader.sdmxml - INFO: Use supplied dsd=… argument for non–structure-specific message


Querying data from ECB's dataflow 'CISS' - Composite Indicator of Systemic Stress...
[Pipeline] ...... (step 1 of 3) Processing augmentation, total=   9.7s
[Pipeline] ............... (step 2 of 3) Processing imp, total=   0.2s
[Pipeline] ............ (step 3 of 3) Processing forest, total=   1.8s


2023-06-18 11:37:58,171 pandasdmx.reader.sdmxml - INFO: Use supplied dsd=… argument for non–structure-specific message


Querying data from ECB's dataflow 'CISS' - Composite Indicator of Systemic Stress...
[CV] END ..augmentation=AugmentSDMX(sources={'ECB': 'CISS'}); total time=  21.1s
Querying data from ECB's dataflow 'CISS' - Composite Indicator of Systemic Stress...


2023-06-18 11:38:07,471 pandasdmx.reader.sdmxml - INFO: Use supplied dsd=… argument for non–structure-specific message


[Pipeline] ...... (step 1 of 3) Processing augmentation, total=  18.3s
[Pipeline] ............... (step 2 of 3) Processing imp, total=   0.4s
[Pipeline] ............ (step 3 of 3) Processing forest, total=   4.4s


2023-06-18 11:38:30,335 pandasdmx.reader.sdmxml - INFO: Use supplied dsd=… argument for non–structure-specific message


Querying data from ECB's dataflow 'CISS' - Composite Indicator of Systemic Stress...
[CV] END ..augmentation=AugmentSDMX(sources={'ECB': 'CISS'}); total time=  41.7s
[Pipeline] ...... (step 1 of 3) Processing augmentation, total=   0.0s
[Pipeline] ............... (step 2 of 3) Processing imp, total=   0.0s
[Pipeline] ............ (step 3 of 3) Processing forest, total=   2.0s


In [ ]:
grid.best_params_

{'augmentation': 'passthrough'}

In [ ]:
print(f"In this particular case, the best model was achieved by {'not ' if grid.best_params_['augmentation'] == 'passthrough' else ''}using the data augmentation.")

In this particular case, the best model was achieved by not using the data augmentation.


In [ ]:
print(f"The last value in the training dataset was {y_train.tail(1).to_numpy()}. The predicted value was {y_pred_grid}, and the actual value was {y_test.to_numpy()}.")

The last value in the training dataset was [5.2244]. The predicted value was [5.236705], and the actual value was [5.279].


### Sources of data

`gingado` seeks to only lists realiable data sources by choice, with a focus on official sources. This is meant to provide users with the trust that their dataset will be complemented by reliable sources. Unfortunately, it is not possible at this stage to include *all* official sources given the substantial manual and maintenance work. `gingado` leverages the existence of the [Statistical Data and Metadata eXchange (SDMX)](https://sdmx.org), an organisation of official data sources that establishes common data and metadata formats, to download data that is relevant (and hopefully also useful) to users.

The function `list_SDMX_sources` returns a list of codes corresponding to the data sources available to provide `gingado` users with data through SDMX.

In [ ]:
from gingado.utils import list_SDMX_sources

In [ ]:
list_SDMX_sources()

['ABS',
 'ABS_XML',
 'BBK',
 'BIS',
 'CD2030',
 'ECB',
 'ESTAT',
 'ILO',
 'IMF',
 'INEGI',
 'INSEE',
 'ISTAT',
 'LSD',
 'NB',
 'NBB',
 'OECD',
 'SGR',
 'SPC',
 'STAT_EE',
 'UNICEF',
 'UNSD',
 'WB',
 'WB_WDI']

You can also see what the available dataflows are. The code below returns a dictionary where each key is the code for an SDMX source, and the values associated with each key are the code and name for the respective dataflows.

In [ ]:
from gingado.utils import list_all_dataflows

In [ ]:
dflows = list_all_dataflows()
dflows

2023-06-18 11:43:54,715 pandasdmx.reader.sdmxml - DEBUG: Truncate sub-microsecond time in <Prepared>
2023-06-18 11:43:58,219 pandasdmx.reader.sdmxml - DEBUG: Truncate sub-microsecond time in <Prepared>
2023-06-18 11:44:19,840 pandasdmx.reader.sdmxml - DEBUG: Truncate sub-microsecond time in <Prepared>
2023-06-18 11:44:20,874 pandasdmx.reader.sdmxml - DEBUG: Truncate sub-microsecond time in <Prepared>
2023-06-18 11:44:25,735 pandasdmx.reader.sdmxml - DEBUG: Truncate sub-microsecond time in <Prepared>
2023-06-18 11:44:26,799 pandasdmx.reader.sdmxml - DEBUG: Truncate sub-microsecond time in <Prepared>


ABS_XML  ABORIGINAL_POP_PROJ                 Projected population, Aboriginal and Torres St...
         ABORIGINAL_POP_PROJ_REMOTE          Projected population, Aboriginal and Torres St...
         ABS_ABORIGINAL_POPPROJ_INDREGION    Projected population, Aboriginal and Torres St...
         ABS_ACLD_LFSTATUS                   Australian Census Longitudinal Dataset (ACLD):...
         ABS_ACLD_TENURE                     Australian Census Longitudinal Dataset (ACLD):...
                                                                   ...                        
UNSD     DF_UNData_UNFCC                                                       SDMX_GHG_UNDATA
WB       DF_WITS_Tariff_TRAINS                                WITS - UNCTAD TRAINS Tariff Data
         DF_WITS_TradeStats_Development                             WITS TradeStats Devlopment
         DF_WITS_TradeStats_Tariff                                      WITS TradeStats Tariff
         DF_WITS_TradeStats_Trade                 

For example, the dataflows from the World Bank are:

In [ ]:
dflows['WB']

DF_WITS_Tariff_TRAINS             WITS - UNCTAD TRAINS Tariff Data
DF_WITS_TradeStats_Development          WITS TradeStats Devlopment
DF_WITS_TradeStats_Tariff                   WITS TradeStats Tariff
DF_WITS_TradeStats_Trade                     WITS TradeStats Trade
Name: dataflow, dtype: object

In [ ]:
#| echo: false
import nbdev; nbdev.nbdev_export()